# Ruteo

Este notebook permite realizar la optimización de todos los pedidos de 1 día de manera directa, sin detallar todo lo realizado.

## Preparación de la Sesión

Cargamos a la sesión de Google Colab el repositorio con los datos y los módulos desarrollados. Este paso **no** es necesario si se corre el notebook de manera local.

In [2]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [3]:
cd ope3-logistica

/content/ope3-logistica


In [4]:
!pip install -e . &> /dev/null

## Módulos

Importamos librerías y módulos propios para llevar a cabo la optimización.

In [5]:
# Importamos las principales librerías de manejo de datos y visualización.
import pandas as pd
import numpy as np
import plotly.express as px

# Importamos el componente principal Ruteo.
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

# Importamos el módulo creado con metaheuristicas.
import logistica.metaheuristicas as mh

## Carga de Datos

Usamos los datos de inputs en `data/inputs/data_inputs.xlsx` con el formato requerido para incorporar los datos de pedidos y camiones al modelo.

### Subir Datos Externos

En caso de trabajar en Google Colab y querer subir datos que no se encuentren en el repositorio se deben correr la siguientes 2 celdas. Si trabajamos localmente directamente podemos agregar los datos a la carpeta de inputs.

In [49]:
# Solo correr esta celda si importamos datos externos de pedidos y camiones.
from google.colab import files
uploaded = files.upload()

Saving data_inputs.xlsx to data_inputs.xlsx


In [50]:
!rm /content/ope3-logistica/data/inputs/data_inputs.xlsx
!mv data_inputs.xlsx /content/ope3-logistica/data/inputs/data_inputs.xlsx

## Optimización

Teniendo los datos de inputs cargados con el formato correspondiente, elegimos el pedido que queremos optimizar en el dropdown y ejecutamos la carga de datos y la optimización.

In [69]:
#@title Seleccionar Pedido
# Hola
pedidos = 'pedidos4' #@param ["pedidos1", "pedidos2", "pedidos3", "pedidos4", "pedidos5", "pedidos6"]

In [70]:
# Guardamos los datos de las hojas del input en dataframes.
df_pedidos = load_inputs("pedidos")
df_camiones = load_inputs("camiones")
df_parametros_ruteo = load_inputs("parametros_ruteo")
df_parametros_mh = load_inputs("parametros_mh")

if df_parametros_mh.random_state[0] == "None":
  random_state = None
else:
  random_state = df_parametros_mh.random_state[0]
  
if df_parametros_mh.max_time[0] == "None":
  max_time = None
else:
  max_time = df_parametros_mh.max_time[0]

# Preparamos los datos para el pedido particular elegido.
df_pedido = preparar_df_pedidos(df_pedidos, pedidos)

# Iniciamos el ruteo con los camiones y pedidos.
ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=df_parametros_ruteo.costo_oportunidad[0], presupuesto=df_parametros_ruteo.presupuesto[0])
# Obtenemos una solución inicial.
ruteo.get_solucion_inicial(mode=df_parametros_mh.sol_inicial_mode[0], random_state=random_state)

# Optimizamos la solución usando los parámetros definidos en los datos de inputs.
best_sol, history = mh.sa(ruteo, 
                          t_inicial=df_parametros_mh.sa_t_inicial[0],
                          t_final=df_parametros_mh.sa_t_final[0], 
                          k=df_parametros_mh.sa_k[0], 
                          iters=df_parametros_mh.sa_iters[0], 
                          temp_mode=df_parametros_mh.sa_temp_mode[0], 
                          max_time=max_time,
                          prob=df_parametros_mh.vecinos_p[0], 
                          random_state=random_state)
print("\n")
# Vemos el resultado de la optmimización.
best_sol.summary_ruteo(time=history.get("time"), iters=history.get("iters"))

100%|██████████| 150/150 [00:05<00:00, 25.21it/s]

,
Carga Total,102.00
Costo Camiones,102000.00
Costo Oportunidad,21000.00
Costo Total,123000.00
Costo Total por tn,1205.88
Ahorro,-1.16
Tiempo,5.96
Iteraciones,3750.00


## Resultados

In [ ]:
# Podemos ver el resumen de la asignación de los camiones con el siguiente método.
best_sol.summary_camiones()

In [71]:
# Podemos ver el resumen de la asignación de pedidos con el siguiente método.
best_sol.summary_pedidos()

,Carga,Asignado,Camion
Pedido A,2,True,6.0
Pedido B,4,True,3.0
Pedido C,2,True,8.0
Pedido D,2,True,2.0
Pedido E,4,True,5.0
Pedido F,8,True,2.0
Pedido G,7,False,NaN
Pedido H,2,True,2.0
Pedido I,4,True,4.0
Pedido J,8,True,9.0


In [ ]:
best_sol.save_results("data/outputs/data_outputs.xlsx")

In [67]:
# Podemos visualizar el proceso de optimización con el siguiente método.
mh.make_history_plots(history)

In [72]:
# Podemos visualizar la mejor solución en la instancia (best_sol) con el siguiente método.
best_sol.plot_solution()